In [9]:
# Import dependencies
import pandas as pd 

In [4]:
# Reference to CSV and reading CSV into Pandas DataFrame
csv_path = "Resources/Behavioral_Risk.csv"
behaviorialrisk_df = pd.read_csv(csv_path)
behaviorialrisk_df.head(10)

,Year,LocationAbbr,LocationDesc,Datasource,PriorityArea1,PriorityArea2,PriorityArea3,PriorityArea4,Category,Topic,...,Data_Value_Footnote,Confidence_Limit_Low,Confidence_Limit_High,CategoryID,TopicID,IndicatorID,BreakoutCategoryID,BreakOutID,LocationID,GeoLocation
0,2011,NV,Nevada,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC03,AGE04,32.0,"(39.493240390000494, -117.07184056399967)"
1,2011,NV,Nevada,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC04,RAC02,32.0,"(39.493240390000494, -117.07184056399967)"
2,2011,NV,Nevada,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC03,AGE01,32.0,"(39.493240390000494, -117.07184056399967)"
3,2011,NV,Nevada,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC04,RAC07,32.0,"(39.493240390000494, -117.07184056399967)"
4,2011,NV,Nevada,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC04,RAC04,32.0,"(39.493240390000494, -117.07184056399967)"
5,2011,NY,New York,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC03,AGE04,36.0,"(42.82700103200045, -75.54397042699964)"
6,2011,NY,New York,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC03,AGE01,36.0,"(42.82700103200045, -75.54397042699964)"
7,2011,OH,Ohio,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC04,RAC04,39.0,"(40.06021014100048, -82.40426005599966)"
8,2011,OH,Ohio,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC04,RAC07,39.0,"(40.06021014100048, -82.40426005599966)"
9,2011,OH,Ohio,BRFSS,None,None,None,None,Cardiovascular Diseases,Coronary Heart Disease,...,Statistically unstable estimates not presented...,NaN,NaN,C1,T4,BR001,BOC03,AGE01,39.0,"(40.06021014100048, -82.40426005599966)"


In [15]:
behaviorialrisk_df.columns

Index(['Year', 'LocationAbbr', 'LocationDesc', 'Datasource', 'PriorityArea1',
       'PriorityArea2', 'PriorityArea3', 'PriorityArea4', 'Category', 'Topic',
       'Indicator', 'Break_Out_Category', 'Break_out', 'Data_Value_Type',
       'Data_Value_Unit', 'Data_Value', 'Data_Value_Footnote_Symbol',
       'Data_Value_Footnote', 'Confidence_Limit_Low', 'Confidence_Limit_High',
       'CategoryID', 'TopicID', 'IndicatorID', 'BreakoutCategoryID',
       'BreakOutID', 'LocationID', 'GeoLocation'],
      dtype='object')

In [18]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark import SparkFiles
from pyspark.sql.functions import col
from pyspark.sql import DataFrameReader

In [11]:
spark = SparkSession \
    .builder \
    .appName('pyspark_demo_app') \
    .config('spark.driver.extraClassPath',
            'postgresql-42.2.9') \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
df = spark.read.format("csv").option("inferSchema", 
           True).option("header", True).load(file_location)

In [12]:
mode = "append"
jdbc_url="jdbc:postgresql://heartsense.cgnk1yoo8fys.us-east-2.rds.amazonaws.com/heartsense"
config = {"user":"root", 
          "password": "Death4bid!", 
          "driver":"org.postgresql.Driver"}

In [ ]:
behaviorialrisk_df.to_sql

In [19]:
# Append DataFrame to active_user table in RDS
behaviorialrisk_df.write.jdbc(url=jdbc_url, table='behavior_risk', mode=mode, properties=config)

AttributeError: 'DataFrame' object has no attribute 'write'

In [ ]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://inspector20jan.s3.us-east-2.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)

user_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("user_data.csv"), inferSchema=True, sep=',')
user_data_df.show(10)

In [ ]:
# Load in user_payment.csv from S3 into a DataFrame

url = "https://inspector20jan.s3.us-east-2.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)

user_payment_df = spark.read.option('header', 'true').csv(SparkFiles.get("user_payment.csv"), inferSchema=True, sep=',')
user_payment_df.show(10)

In [ ]:
# Join the two DataFrame
joined_df= user_data_df.join(user_payment_df, on="username", how="inner")
joined_df.show()

In [ ]:
# Drop null values
dropna_df = joined_df.dropna()
dropna_df.show()

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
cleaned_df = dropna_df.filter(col("active_user")  == True)
cleaned_df.show()

In [ ]:
# Create user dataframe to match active_user table
clean_user_df = cleaned_df.select(["id", "first_name", "last_name", "username"])
clean_user_df.show()

In [17]:
my_df = spark.read.jdbc(url=jdbc_url, table='doctors', properties=config)
my_df.limit(10).show()

+---+-----------+---------------+
| id| speciality|taking_patients|
+---+-----------+---------------+
|  1| cardiology|           true|
|  2|orthopedics|          false|
|  3| pediatrics|           true|
+---+-----------+---------------+



In [ ]:
# Append DataFrame to active_user table in RDS

clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)